<a href="https://colab.research.google.com/github/KenzaB27/segmentation-of-pelvic-bone/blob/main/src/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup

In [1]:
!nvidia-smi

Wed May 26 19:22:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/KenzaB27/segmentation-of-pelvic-bone.git
!pip install SimpleITK
%cd /content/segmentation-of-pelvic-bone/src

Cloning into 'segmentation-of-pelvic-bone'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 177 (delta 95), reused 128 (delta 55), pack-reused 0
Receiving objects: 100% (177/177), 3.11 MiB | 29.26 MiB/s, done.
Resolving deltas: 100% (95/95), done.
     |████████████████████████████████| 47.4MB 106kB/s 
/content/segmentation-of-pelvic-bone/src


In [4]:
!git pull

Already up to date.


In [5]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import SimpleITK as sitk
import analysis as ana
import numpy as np 
import importlib

import tensorflow as tf 
tfk = tf.keras
tfkl = tfk.layers

In [6]:
PATH = "/content/drive/My Drive/KTH/HL2027/"
DATA_PATH = PATH +'data/'

## Load Data

In [7]:
%%time
importlib.reload(ana)
data = ana.PelvicData(root_path=PATH)

CPU times: user 13.6 s, sys: 4.08 s, total: 17.7 s
Wall time: 34.8 s


In [8]:
data.cmn_img_3c[42].shape

(291, 512, 512, 3)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(data.X_train_3C, data.y_train, test_size=0.15, random_state=42, stratify=data.y_train)
print(f"X train shape:{X_train.shape}, y train shape:{y_train.shape} ")
print(f"X val shape:{X_val.shape}, y val shape:{y_val.shape} ")

X train shape:(729, 512, 512, 3), y train shape:(729,) 
X val shape:(129, 512, 512, 3), y val shape:(129,) 


## ResNet V2

In [19]:
input = tfkl.Input([None, None, 3], dtype = tf.int16)
x = tf.cast(input, tf.float32)
x = tfk.applications.resnet_v2.preprocess_input(x)
resnet_model = tfk.applications.ResNet50V2(weights="imagenet", include_top=False, pooling="max")
resnet_output = resnet_model(x)
hidden_layer = tfkl.Dense(256, activation='relu')(resnet_output)
hl_reg = tfkl.Dropout(0.2)(hidden_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hl_reg)
model = tf.keras.models.Model(inputs=input, outputs=output_layer)

In [20]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
tf.cast_2 (TFOpLambda)       (None, None, None, 3)     0         
_________________________________________________________________
tf.math.truediv_2 (TFOpLambd (None, None, None, 3)     0         
_________________________________________________________________
tf.math.subtract_2 (TFOpLamb (None, None, None, 3)     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
dense_4 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0   

### Training 

In [21]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
resnet_model.trainable = False
for layer in resnet_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-3)
model.compile(optimizer=optimizer, loss='binary_crossentropy', 
              metrics=['acc'])

In [ ]:
checkpoint_filepath = PATH + '/checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
history = model.fit(x=X_train, y=y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)

Epoch 1/50
23/23 [==============================] - 11s 307ms/step - loss: 14.3913 - acc: 0.7805 - val_loss: 4.2220 - val_acc: 0.8837
Epoch 2/50
23/23 [==============================] - 6s 260ms/step - loss: 9.7903 - acc: 0.8313 - val_loss: 1.8385 - val_acc: 0.8915
Epoch 3/50
23/23 [==============================] - 6s 262ms/step - loss: 7.5004 - acc: 0.8203 - val_loss: 0.9530 - val_acc: 0.9070
Epoch 4/50
23/23 [==============================] - 6s 259ms/step - loss: 5.5431 - acc: 0.8436 - val_loss: 0.1619 - val_acc: 0.9767
Epoch 5/50
23/23 [==============================] - 6s 259ms/step - loss: 4.3883 - acc: 0.8711 - val_loss: 0.1328 - val_acc: 0.9690
Epoch 6/50
23/23 [==============================] - 6s 259ms/step - loss: 3.1032 - acc: 0.8889 - val_loss: 2.0639 - val_acc: 0.8837
Epoch 7/50
23/23 [==============================] - 6s 260ms/step - loss: 4.4991 - acc: 0.8697 - val_loss: 0.4102 - val_acc: 0.9457
Epoch 8/50
23/23 [==============================] - 6s 262ms/step - loss: 

In [ ]:
plt.figure()
plt.plot(history.history["loss"], label="training loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.title("Loss for Inception model")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["acc"], label="training accuracy")
plt.plot(history.history["val_acc"], label="validation accuracy")
plt.title("Accuracy for Inception model")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Test Performance on Common Images

In [ ]:
%%time
for id, cmn_img in data.cmn_img_3c.items():
  y_pred = model.predict(cmn_img)
  argmax = np.argmax(y_pred)
  plt.figure()
  
  plt.subplot(121)
  plt.plot(y_pred)
  plt.title(f"Probability for case {id}")

  plt.subplot(122)
  plt.imshow(data.cmn_img[id][argmax], cmap="gray")
  plt.show()